<a href="https://colab.research.google.com/github/canamac/zeka-can/blob/main/DARALMA_CO%C4%B0N_TARAMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
from concurrent.futures import ProcessPoolExecutor, as_completed
import warnings
from datetime import datetime, timedelta
import time
import os

warnings.filterwarnings('ignore')
os.environ['YFINANCE_CACHE_OFF'] = '1'  # Cache'i devre dışı bırak

class CoinScanner:
    def __init__(self, parameters=None):
        self.default_params = {
            'zigzag_high_period': 10,
            'zigzag_low_period': 10,
            'min_movement_percent': 0.16,
            'ma_period': 20,
            'std_dev_multiplier': 2.0,
            'daralma_esik': 0.5,
            'ma_length': 10,
            'volume_length': 7,
            'max_retries': 3,
            'retry_delay': 1  # Yeniden deneme aralığı (saniye)
        }
        self.params = parameters or self.default_params

    def _validate_data(self, df):
        """Veri bütünlüğünü kapsamlı şekilde kontrol eder"""
        required_columns = ['Open', 'High', 'Low', 'Close', 'Volume']
        if df.empty:
            raise ValueError("Boş veri çerçevesi")
        if not all(col in df.columns for col in required_columns):
            raise ValueError(f"Eksik sütunlar. Mevcut sütunlar: {df.columns.tolist()}")
        if not isinstance(df.index, pd.DatetimeIndex):
            raise ValueError("Geçersiz indeks tipi")
        if len(df) < self.params['ma_period']:
            raise ValueError(f"Yetersiz veri: {len(df)} çubuk")
        return df

    def _ensure_datetime_index(self, df):
        """DatetimeIndex kontrolü ve düzeltmesi"""
        if not isinstance(df.index, pd.DatetimeIndex):
            if 'Date' in df.columns:
                df.index = pd.to_datetime(df['Date'])
                df = df.drop('Date', axis=1)
            else:
                raise ValueError("Tarih sütunu bulunamadı")
        return df

    def _calculate_coin_signals(self, coin_data):
        try:
            df = coin_data.copy()

            # 1. Veri doğrulama ve indeks düzeltme
            df = self._ensure_datetime_index(df)
            self._validate_data(df)

            # 2. Teknik göstergeler
            df['ma'] = df['Close'].rolling(self.params['ma_period']).mean()
            df['std_dev'] = df['Close'].rolling(self.params['ma_period']).std()
            df['upper_band'] = df['ma'] + self.params['std_dev_multiplier'] * df['std_dev']
            df['lower_band'] = df['ma'] - self.params['std_dev_multiplier'] * df['std_dev']

            # 3. ZigZag hesaplamaları
            df['zigzag_high'] = df['High'].rolling(self.params['zigzag_high_period']).max() == df['High']
            df['zigzag_low'] = df['Low'].rolling(self.params['zigzag_low_period']).min() == df['Low']

            # 4. Sinyal koşulları
            min_move = df['Close'] * self.params['min_movement_percent'] / 100
            df['LongMumu'] = (df['zigzag_high'] &
                            (df['High'] > df['High'].shift(self.params['zigzag_high_period'])) &
                            (df['Close'] - df['Close'].shift(self.params['zigzag_high_period']) >= min_move))

            df['ShortMumu'] = (df['zigzag_low'] &
                             (df['Low'] < df['Low'].shift(self.params['zigzag_low_period'])) &
                             (df['Close'].shift(self.params['zigzag_low_period']) - df['Close'] >= min_move))

            # 5. Son sinyaller
            daralma_condition = (df['upper_band'] - df['lower_band']).iloc[-1] < self.params['daralma_esik']
            last_buy = df['LongMumu'].iloc[-5:].any()
            last_sell = df['ShortMumu'].iloc[-5:].any()

            return {
                'BUY': bool(last_buy and daralma_condition),
                'SELL': bool(last_sell and daralma_condition),
                'TRADE': bool((last_buy or last_sell) and daralma_condition),
                'timestamp': df.index[-1].strftime('%Y-%m-%d %H:%M')
            }

        except Exception as e:
            print(f"Hesaplama hatası: {str(e)}")
            print(f"Veri şekli: {df.shape if 'df' in locals() else 'Veri yok'}")
            return None

    def download_data(self, coin):
        """Gelişmiş veri indirme fonksiyonu"""
        for attempt in range(self.params['max_retries']):
            try:
                # Tarih aralığını ayarla
                end_date = datetime.now()
                start_date = end_date - timedelta(days=self.params.get('days', 59))

                # YFinance alternatif indirme yöntemi
                ticker = yf.Ticker(coin)
                data = ticker.history(
                    start=start_date.strftime('%Y-%m-%d'),
                    end=end_date.strftime('%Y-%m-%d'),
                    interval=self.params.get('timeframe', '1d'),
                    actions=False,
                    auto_adjust=True
                )

                if not data.empty:
                    data = data.tz_localize(None)
                    data = self._ensure_datetime_index(data)
                    return coin, data

            except Exception as e:
                print(f"İndirme hatası ({coin} Deneme {attempt+1}/3): {str(e)}")
                time.sleep(self.params['retry_delay'])

        print(f"{coin} verisi alınamadı")
        return None

    def scan_coins(self, coin_list, timeframe='1d', days=59, parallel=True):
        """Optimize edilmiş tarama fonksiyonu"""
        self.params.update({'timeframe': timeframe, 'days': days})
        results = {}

        if parallel:
            with ProcessPoolExecutor(max_workers=2) as executor:  # Worker sayısını azalt
                futures = {executor.submit(self.download_data, coin): coin for coin in coin_list}
                for future in as_completed(futures):
                    coin = futures[future]
                    try:
                        result = future.result()
                        if result:
                            coin_name, data = result
                            signal = self._calculate_coin_signals(data)
                            if signal:
                                results[coin_name] = signal
                    except Exception as e:
                        print(f"İşleme hatası ({coin}): {str(e)}")
        else:
            for coin in coin_list:
                try:
                    result = self.download_data(coin)
                    if result:
                        coin_name, data = result
                        signal = self._calculate_coin_signals(data)
                        if signal:
                            results[coin_name] = signal
                except Exception as e:
                    print(f"İşleme hatası ({coin}): {str(e)}")

        return results

if __name__ == "__main__":
    scanner = CoinScanner()

    try:
        signals = scanner.scan_coins(
            coin_list=['BTC-USD', 'ETH-USD', 'ADA-USD', 'DOT-USD', 'SOL-USD'],
            timeframe='1d',
            days=59  # 60 gün yerine 59 kullan
        )

        print("\nSon Sinyaller:")
        for coin, data in signals.items():
            print(f"{coin} [{data['timestamp']}]")
            print(f"► BUY: {'✅' if data['BUY'] else '❌'}")
            print(f"► SELL: {'✅' if data['SELL'] else '❌'}")
            print(f"► TRADE: {'✅' if data['TRADE'] else '❌'}")
            print("-"*40)

    except Exception as e:
        print(f"Kritik hata: {str(e)}")


Son Sinyaller:
ETH-USD [2025-01-29 00:00]
► BUY: ❌
► SELL: ❌
► TRADE: ❌
----------------------------------------
BTC-USD [2025-01-29 00:00]
► BUY: ❌
► SELL: ❌
► TRADE: ❌
----------------------------------------
ADA-USD [2025-01-29 00:00]
► BUY: ❌
► SELL: ✅
► TRADE: ✅
----------------------------------------
DOT-USD [2025-01-29 00:00]
► BUY: ❌
► SELL: ❌
► TRADE: ❌
----------------------------------------
SOL-USD [2025-01-29 00:00]
► BUY: ❌
► SELL: ❌
► TRADE: ❌
----------------------------------------
